#  Model Wrapper


In [ ]:

import torch
import torch.nn as nn
from torch.testing import assert_close
torch.manual_seed(42)
import tool
from tqdm import tqdm


### Config

In [ ]:
IS_SKIP_TEST =False

GPT_CONFIG = {
    "num_epochs":10,
    "batch_size":4,
    "vocab_size": 50257,     # 词汇表大小
    "context_len": 256,  # 上下文长度
    "emb_dim": 512,          # 嵌入维度
    "n_heads": 8,           # 注意力头的数量
    "n_layers": 12,          # 层数
    "drop_rate": 0.1,        # dropout率
    "qkv_bias": False ,      # 查询-键-值偏置
}

TOKEN_TYPE="gpt2"


### Set device to (type='cuda')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:

class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
    
    def forward(self,x):
        return x
    
class DummyLayerNorm(nn.Module):
    def __init__(self, norm_shape,eps=1e-5):
        super().__init__()
        
    def forward(self,x):
        return x
        

class DummyGPT(nn.Module):
    def __init__(self,cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg['vocab_size'],cfg['emb_dim']) #  “字典表”  (vocab_size, emb_dim) vocab_size 行，每一行对应一个 token 的emb_dim维的向量 
        self.pos_emb = nn.Embedding(cfg['context_len'],cfg['emb_dim']) # (context_len, emb_dim)
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        self.trf_blocks =  nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg['emb_dim'],cfg['vocab_size'],bias=False 
        )# y = x · W^T + b   W的形状为[vocab_size,emb_dim] 本质是计算x与W的相似度 ，得到vocab_size个y向量
       
    def forward(self,in_idx):
        #in_idx 通常是一个整数张量（Tensor），形状一般为 (batch_size, seq_len)
        batch_size, seq_len = in_idx.shape  #in_idx 每个元素都是 token 的索引（范围是 [0, vocab_size-1])
        tok_embeds = self.tok_emb(in_idx) #查“字典表”映射  嵌入向量(batch_size, seq_len)-->(batch_size, seq_len, emb_dim) 

        pos_embeds = self.pos_emb(torch.arange(seq_len,device=in_idx.device))  #生成一个从 0 到 seq_len-1 的整数序列 (seq_len,) -->(seq_len, emb_dim)
   
        x = tok_embeds + pos_embeds #pos_embeds会自动广播为 -->(batch_size, seq_len, emb_dim)
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x) #-->(batch_size, seq_len, emb_dim)
        logits = self.out_head(x) #(batch_size, seq_len, emb_dim)-->(batch_size, seq_len, vocab_size)
        return logits
        
        

In [ ]:
@tool.skip_execution(skip=IS_SKIP_TEST)
def test_dummyModel():
    model = DummyGPT(GPT_CONFIG)
    return model

test_dummyModel()

## Define layerNorm

In [ ]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))
        
    def forward(self,x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim =-1 ,keepdim =True, unbiased =False)
        norm_x = (x-mean)/torch.sqrt(var+self.eps)
        return self.scale*norm_x + self.shift

### test layerNorm

In [ ]:
@tool.skip_execution(skip=IS_SKIP_TEST)
def test_layer_norm():
    batch_size = 2
    seq_len = 5
    emb_dim = 3  
    x = torch.randn(batch_size, seq_len, emb_dim)  # 随机生成输入张量
    
    custom_ln = LayerNorm(emb_dim)
    official_ln = nn.LayerNorm(emb_dim, eps=1e-5, elementwise_affine=True)
    

    official_ln.weight.data.copy_(custom_ln.scale.data)
    official_ln.bias.data.copy_(custom_ln.shift.data)
 
    custom_out = custom_ln(x)
    official_out = official_ln(x)
    print(custom_out)
    print(official_out)

    assert_close(
        custom_out, 
        official_out, 
        rtol=1e-5,  # 相对误差容忍度
        atol=1e-5   # 绝对误差容忍度
    )
    print("自定义LayerNorm与官方实现输出一致")

test_layer_norm()

## Define activate function

Φ(x) ≈ 0.5 * (1 + tanh(√(2/π) * (x + 0.044715 * x³)))

In [ ]:
class GELU(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def forward(self,x):
        return 0.5*x*(1+ 
                      torch.tanh(torch.sqrt(torch.tensor(2/torch.pi))
                                 *(x+0.044715*torch.pow(x,3))
                                )
                      )
        


### test gelu

In [ ]:
@tool.skip_execution(skip=IS_SKIP_TEST)
def test_gelu():
    x = torch.tensor([-3.0, -1.0, 0.0, 0.5, 1.0, 2.0, 5.0])
    
    custom_gelu = GELU()
    official_gelu = nn.GELU()

    custom_out = custom_gelu(x)
    official_out = official_gelu(x)
    
    # 打印结果进行直观对比
    print("输入值:", x)
    print("自定义GELU输出:", custom_out)
    print("官方GELU输出:", official_out)
 
    assert_close(
        custom_out,
        official_out,
        rtol=1e-3,  # 相对误差容忍度
        atol=1e-3   # 绝对误差容忍度
    )
    print("\n自定义GELU与官方实现近似一致")
    
test_gelu()

## Define FFN
通过两层线性变换和激活函数，对注意力机制输出的特征进行非线性加工，增强模型表达能力。

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            #中间层hidden_dim通常设为4*emb_dim（如原始 Transformer 中为 512→2048→512），通过扩展维度捕捉更丰富的特征
            nn.Linear(cfg['emb_dim'],4*cfg['emb_dim']),  
            GELU(),
            nn.Dropout(cfg['drop_rate']),
            nn.Linear(4*cfg['emb_dim'],cfg['emb_dim'])
        )
        
    def forward(self,x):
        return self.layers(x)

## Define MultiAttention

In [ ]:
class CausalAttention(nn.Module):
    def __init__(self, d_in, d_out,context_len,dropout,qkv_bias=False):
        super().__init__()
        self.d_out =d_out
        self.W_q = nn.Linear(d_in,d_out,bias= qkv_bias)
        self.W_k = nn.Linear(d_in,d_out,bias= qkv_bias)
        self.W_v = nn.Linear(d_in,d_out,bias= qkv_bias)
        self.dropout = nn.Dropout(dropout)
        #缓冲区（buffer）是模型中不需要被训练的参数（与 nn.Parameter 不同，后者是可学习参数），但会随模型一起保存（state_dict 中包含）
        self.register_buffer(
            'mask', 
            torch.triu(torch.ones(context_len,context_len),diagonal=1)
        )
    
    def forward(self,x):
        b,num_tokens,d_in = x.shape
        keys = self.W_k(x)
        queries = self.W_q(x)
        values = self.W_v(x)
        
        att_score = queries @ keys.transpose(1,2)
        att_score.masked_fill_(self.mask.bool()[:num_tokens,:num_tokens],-torch.inf) # 上面的register_buffer  形状为 (num_tokens, num_tokens) 的子矩阵
        att_weight = torch.softmax(att_score/keys.shape[-1]**0.5, dim=-1)
        att_weight = self.dropout(att_weight)
        context_vec = att_weight @ values
        return context_vec
        
class MultiHeadAttendtion(nn.Module):
    def __init__(self, d_in, d_out,context_len,dropout,num_heads,qkv_bias=False):
        super().__init__()
        # ModuleList与nn.Sequential不同，它不自动执行前向传播，而是需要手动遍历调用，适合需要单独处理每个子模块的场景
        self.heads = nn.ModuleList(
            [CausalAttention(d_in,d_out,context_len,dropout,qkv_bias) for _ in range(num_heads)]
        )
        
    def forward(self,x):
        return torch.cat([head(x) for head in self.heads],dim=-1)

In [ ]:
# TODO 更高效的MutiAttention 减少计算量

#参数规模更小（d_model×d_model 对比 num_heads×d_model×head_dim)
class MultiHeadAttendtion_new(nn.Module):
    def __init__(self, d_in, d_out,context_len,dropout,num_heads,qkv_bias=False):
        super().__init__()
        self.d_out =d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        self.W_q = nn.Linear(d_in,d_out,bias= qkv_bias)
        self.W_k = nn.Linear(d_in,d_out,bias= qkv_bias)
        self.W_v = nn.Linear(d_in,d_out,bias= qkv_bias)
        self.out_proj =nn.Linear(d_out,d_out) # out_proj 可以学习如何 “融合” 这些头的信息（例如对不同头的特征赋予不同权重），而不是简单保留原始拼接结果
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask', 
            torch.triu(torch.ones(context_len,context_len),diagonal=1)
        )
    
    def forward(self,x):
        b,num_tokens,d_in = x.shape
        keys = self.W_k(x)
        queries = self.W_q(x)
        values = self.W_v(x)
        
        keys = keys.view(b,num_tokens,self.num_heads,self.head_dim)
        queries = queries.view(b,num_tokens,self.num_heads,self.head_dim)
        values = values.view(b,num_tokens,self.num_heads,self.head_dim)
        
        #(b,num_tokens,num_heads,head_dim) --> (b,num_heads,num_tokens,head_dim)   
        keys = keys.transpose(1,2)
        queries = queries .transpose(1,2)
        values = values.transpose(1,2)
        
        
        att_score = queries @ keys.transpose(2,3)
        att_score.masked_fill_(self.mask.bool()[:num_tokens,:num_tokens],-torch.inf)
        att_weight = torch.softmax(att_score/keys.shape[-1]**0.5, dim=-1)
        att_weight = self.dropout(att_weight)
        context_vec = (att_weight @ values).transpose(1,2)
        context_vec = context_vec.contiguous().view(b,num_tokens,self.d_out)
        context_vec = self.out_proj(context_vec)
        return context_vec

## Define Transformer block

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttendtion_new(
            d_in= cfg["emb_dim"],
            d_out= cfg['emb_dim'],
            context_len=  cfg['context_len'],
            num_heads= cfg["n_heads"],
            dropout= cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"]
        )
        self.ff =FeedForward(cfg)
        self.norm1 = LayerNorm(cfg['emb_dim']) #norm1：用于注意力模块（self.att）的输入归一化
        self.norm2 = LayerNorm(cfg['emb_dim']) #norm2：用于前馈网络（self.ff）的输入归一化
        self.dropout = nn.Dropout(cfg['drop_rate'])
        
    
    def forward(self,x):
       # 注意力分支：LayerNorm -> 注意力 -> Dropout -> 残差连接
        x = x + self.dropout(self.att(self.norm1(x))) 
        # FFN分支：LayerNorm -> FFN -> Dropout -> 残差连接
        x = x + self.dropout(self.ff(self.norm2(x)))  
        return x
        
         


## Tokenizer

In [ ]:
# ! pip install tiktoken

In [ ]:
import tiktoken


def text_to_tokenIds(text,tokenizer):
    encoded = tokenizer.encode(text,allowed_special={'<|endoftext|>'})
    encoded_tensor =torch.tensor(encoded).unsqueeze(0)
    return encoded_tensor

def tokenIds_to_text(token_ids,tokenizer):
    flat =token_ids.squeeze(0)
    return tokenizer.decode(flat.tolist())


## Generate text

max_new_tokens: 往后生成n个token

context_size: 更关注最近的上下文，只取size数量的token

In [ ]:

def  generate_text_greedy(model,idxs,max_new_tokens,context_size):
    model.eval()
    for _ in range(max_new_tokens):
        idx_condition = idxs[:,-context_size:]
        with torch.no_grad():
            logits = model(idx_condition)
            
        #生成时：只需要最后一个位置的 logits
        logits = logits[:,-1,:]
        probas =torch.softmax(logits,dim=-1)
        idx_next = torch.argmax(probas,dim=-1,keepdim=True)
        idxs = torch.cat((idxs,idx_next),dim=1)
    return idxs

def  generate_text_withsample(model,idxs,max_new_tokens,context_size,
                              temperture=0.0,top_k=None,eos_id=None):
    model.eval()
    for _ in range(max_new_tokens):
        idx_condition = idxs[:,-context_size:]
        
        with torch.no_grad():
            logits = model(idx_condition)
        logits = logits[:,-1,:]
        
        if top_k is not None:
            top_logits,_ = torch.topk(logits,top_k)
            min_val = top_logits[:,-1]
            logits = torch.where(logits<min_val,torch.tensor(float('-inf')).to(logits.device),logits)
            
        if temperture > 0:
            probas =torch.softmax(logits/temperture,dim=-1)
            idx_next = torch.multinomial(probas,num_samples=1) # 高温度下，概率分布平缓，采样会更大概率选中次优选项
        else:
            probas =torch.softmax(logits,dim=-1)
            idx_next = torch.argmax(probas,dim=-1,keepdim=True)# greedy 
        if idx_next == eos_id:
            break
        idxs = torch.cat((idxs,idx_next),dim=1)
    return idxs

### test gernerate

In [ ]:
@tool.skip_execution(skip=IS_SKIP_TEST)
def test_tokenizer():
    model =DummyGPT(GPT_CONFIG)
    test_context ="今天的天气是晴天，适合出去走走"
    #test_context = "I like the weather"
    print(f'{test_context}--ori')
    tokenizer =tiktoken.get_encoding(TOKEN_TYPE)
    tokenids =text_to_tokenIds(test_context,tokenizer)
    print(f'{tokenIds_to_text(tokenids,tokenizer)}--recover') 


    tokenids_g = generate_text_greedy(model,tokenids,max_new_tokens=10,context_size=GPT_CONFIG['context_len'])

    print(f'{tokenIds_to_text(tokenids_g,tokenizer)}--greedy') 
    
    tokenids_s = generate_text_withsample(model,tokenids,max_new_tokens=10,context_size=GPT_CONFIG['context_len'],
                                        temperture=0.5,top_k=50,eos_id=None)

    print(f'{tokenIds_to_text(tokenids_s,tokenizer)}--sample') 

test_tokenizer()


Epoch 过程中查看生成的文本

查看模型生成的新 token 数量（max_new_tokens）:
* 训练监控（最常用）：20-50 个 token
* 轻量化验证（追求效率）：10-20 个 token
* 深度观察（关键节点）：50-100 个 token

In [ ]:

def generate_and_print(model,tokenizer,device,start_context,max_new_tokens, temperture=0,top_k=None,eos_id=None):
    model.eval()
    context_size = model.pos_emb.weight.shape[0]
    encoded = text_to_tokenIds(start_context,tokenizer).to(device)
    with torch.no_grad():
        token_ids = generate_text_withsample(model,idxs=encoded,max_new_tokens=max_new_tokens,context_size=context_size, temperture=temperture,top_k=top_k,eos_id=eos_id)
        decoded_text = tokenIds_to_text(token_ids,tokenizer)
    print(decoded_text.replace("\n"," "))
    model.train()
        

## GPTDataLoader

In [ ]:

from torch.utils.data import DataLoader,Dataset

class GPTDataset(Dataset):
    def __init__(self, texts: list[str], tokenizer, max_len: int, stride: int):
        super().__init__()
        self.input_ids = []
        self.target_ids = []
        self.max_len = max_len
        self.stride = stride
        
        for idx, text in enumerate(tqdm(texts, desc="Process text")):
            
            if not isinstance(text, str):
                raise TypeError(f"The type of the {idx}-th element is {type(text)}")
            
            if not text.strip():
                continue
            
            # encode single text
            tokenids = tokenizer.encode(text)
            token_len= len(tokenids)
            # print('token len:',token_len)
            
            if token_len < max_len + 1:
                continue  # 连一个完整样本都无法生成，直接跳过
            
            # 计算该文本可生成的样本数
            max_start = token_len - max_len - 1 # 最后一个有效起始位置
            num_samples = (max_start// stride) + 1 if max_start >= 0 else 0
            
            if num_samples > 0:
                # 滑动窗口生成样本
                for i in range(0, max_start, stride):
                    input_chunk = tokenids[i:i+max_len]
                    target_chunk = tokenids[i+1:i+max_len+1]  # 目标是输入的下一个token
                    if len(target_chunk) < max_len:
                            continue  # 跳过不完整的目标
                    self.input_ids.append(torch.tensor(input_chunk))
                    self.target_ids.append(torch.tensor(target_chunk))
        
        print(f"Total samples: {len(self.input_ids)}")
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        try:
            input_batch = self.input_ids[idx]
            target_batch =self.target_ids[idx]
            return input_batch,target_batch
        except Exception as e:
            print(f'Failed to load{idx}:{str(e)}')
            raise
    

'''
DataLoader 本质是一个批次生成器迭代索引：
自动生成从 0 到 len(dataset)-1 的索引，通过 dataset.__getitem__(idx) 逐个获取样本
'''
def GPTDataloader(txts:list[str],token_type,batch_size=4,max_len=246,stride=128,shuffle=True,drop_last=True,num_works=0):
    tokenizer =tiktoken.get_encoding(token_type)
    ds = GPTDataset(txts,tokenizer,max_len,stride)
    dl = DataLoader(
        ds,
        batch_size =batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_works
    )
    return dl

## Loss funcion

In [ ]:
def calc_loss_batch(input_batch,target_batch,model,device):
    input_batch = input_batch.to(device)
    target_batch = target_batch.to(device)
    logits = model(input_batch) # 隐式调用 model.forward(input_batch)
    loss =torch.nn.functional.cross_entropy(logits.flatten(0,1),target_batch.flatten())
    return loss

#快速验证：指定 num_batchs=n，只跑前n个批次，节省时间。
def calc_loss_loader(data_loader,model,device,num_batchs=None):
    total_loss = 0
    total_batchs =len(data_loader)
    # print('total batch count:' ,total_batchs)
    if  total_batchs == 0:
        return float('nan')
    elif num_batchs is None:
        num_batchs = total_batchs 
    else:
        num_batchs = min(num_batchs,total_batchs)
    
    for i ,(input_batch,target_batch) in enumerate(data_loader):# dataset.__getitem__(idx)
        if i < num_batchs:
            loss = calc_loss_batch(input_batch,target_batch,model,device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batchs

def evaluate_model(model,train_loader,valid_loader,device,eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader,model,device,num_batchs=eval_iter)
        valid_loss = calc_loss_loader(valid_loader,model,device,num_batchs=eval_iter)
    model.train()
    return train_loss,valid_loss

### test loss function

In [ ]:
@tool.skip_execution(skip=IS_SKIP_TEST)
def test_loss():
    model = DummyGPT(GPT_CONFIG)
    model.to(device)
    file_path ="../datasets/the-verdict.txt"
    with open (file_path,"r",encoding="utf-8") as file:
        text_data =file.read()
        
    split_idx = int(0.8*len(text_data))
    train_data = text_data[:split_idx]
    print(len(train_data))
    
    train_loader = GPTDataloader(
        [train_data],
        TOKEN_TYPE,
        batch_size = GPT_CONFIG['batch_size'],
        max_len = GPT_CONFIG["context_len"],
        stride = GPT_CONFIG["context_len"] // 2, 
        drop_last=True,
        shuffle= True, 
        num_works=0   
    )

    return calc_loss_loader(train_loader,model,device=device)

test_loss()

### Save model

In [ ]:
def savemodel(path,model,optimizer):
    torch.save({
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        },
        path
    )